In [37]:
from dask.distributed import Client, progress, get_worker
client = Client("tcp://131.180.106.138:8786")

In [38]:
file_list = [
    'http://www.nwhiker.com/wallpaper/wp01.jpg',
    'http://www.nwhiker.com/wallpaper/wp02.jpg',
#     'http://www.nwhiker.com/wallpaper/wp03.jpg',
#     'http://www.nwhiker.com/wallpaper/wp04.jpg',
#     'http://www.nwhiker.com/wallpaper/wp05.jpg',
#     'http://www.nwhiker.com/wallpaper/wp06.jpg',
#     'http://www.nwhiker.com/wallpaper/wp07.jpg',
#     'http://www.nwhiker.com/wallpaper/wp08.jpg',
#     'http://www.nwhiker.com/wallpaper/wp09.jpg',
#     'http://www.nwhiker.com/wallpaper/wp10.jpg'
]

In [39]:

def run_in_worker(file_name):
    import requests
    import os
    from xfZlibWrapper import xfZlibWrapper

    # Download to filesystem
    print(f"Downloading file from URL {file_name}")
    os.makedirs(os.path.dirname("data/"), exist_ok=True)    
    file_path = 'data/' + file_name.split("/")[-1]
    r = requests.get(file_name)
    open(file_path, 'wb').write(r.content)
    
    
    # Call FPGA driver
    print("calling constructor")
    xfZlib = xfZlibWrapper(b"/shares/bulk/shashankaggarw/compress_decompress.xclbin")
    
    size = xfZlib.compress_file(file_path, file_path + ".gz")
    print(f'Compressed from {os.path.getsize(file_path)} to {size} bytes')
    return size


In [40]:
# distributed_data = client.scatter(file_list)
# print(len(distributed_data))
from datetime import datetime

t0 = datetime.now()
futures = client.map(run_in_worker,  file_list)
print("Dispatched work to worker(s)")

results = client.gather(futures)
t1 = datetime.now()
print("Received results:")
print(results)
print("Time taken:")
print(t1-t0)

Dispatched work to worker(s)
Received results:
[306724, 334821]
Time taken:
0:00:10.221838


distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
asyncio.